In [31]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='.env.local')
import os
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
import requests
from bs4 import BeautifulSoup
import time
import random

In [13]:
#existing_indexes = pc.list_indexes()
# if "rag" not in existing_indexes:
#     pc.create_index(
#         name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
#     )
# else:
#     print("Index 'rag' already exists.")


In [19]:
# from pinecone.exceptions import PineconeApiException
# api_key=os.getenv("PINECONE_API_KEY")
# pc = Pinecone(api_key)


# try:
#     existing_indexes = pc.list_indexes()
#     if "rag" not in [idx['name'] for idx in existing_indexes['indexes']]:
#         pc.create_index(
#             name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
#         )
#         print("Index 'rag' created successfully.")
#     else:
#         print("Index 'rag' already exists, skipping creation.")
# except PineconeApiException as e:
#     if e.status == 409:
#         print("Index 'rag' already exists, no action taken.")
#     else:
#         raise e




In [42]:
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from pinecone.exceptions import PineconeApiException
import os

# Load environment variables
load_dotenv()

# Initialize Pinecone
api_key = "b06218ed-b4ea-48db-a318-6e6870d7edd0"
pc = Pinecone(api_key)

try:
    # List existing indexes
    existing_indexes = pc.list_indexes()
    print("Existing indexes:", existing_indexes)

    # Check if the index already exists
    index_names = [idx['name'] for idx in existing_indexes.get('indexes', [])]
    if "rag" not in index_names:
        pc.create_index(
            name="rag",
            dimension=768,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        print("Index 'rag' created successfully.")
    else:
        print("Index 'rag' already exists, skipping creation.")
except PineconeApiException as e:
    # Print the exception details for debugging
    print(f"An error occurred: {e}")
    # Check if the status attribute exists and handle it
    if hasattr(e, 'status') and e.status == 409:
        print("Index 'rag' already exists, no action taken.")
    else:
        raise e


Existing indexes: {'indexes': []}
Index 'rag' created successfully.


In [8]:
# import json
# data = json.load(open("reviews.json"))
# data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Smith is an excellent professor who explains concepts very clearly.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Dr. Johnson is very knowledgeable but can be a bit fast-paced.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Physics',
  'stars': 3,
  'review': 'The lectures are okay, but the exams are quite difficult.'},
 {'professor': 'Dr. Sarah Davis',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Amazing professor! Her labs are very well organized.'},
 {'professor': 'Dr. James Wilson',
  'subject': 'History',
  'stars': 2,
  'review': 'Not very engaging, and the material is dry.'},
 {'professor': 'Dr. Linda Garcia',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Dr. Garcia is great, but her grading is tough.'},
 {'professor': 'Dr. Robert Martinez',
  'subject': 'Political Science',
  'stars': 3,
  'review': 'Interesting lect

In [33]:
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

def scrape_ratemyprofessors(num_pages=5):
    base_url = "https://www.ratemyprofessors.com/search/teachers?query=*&sid=1073"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    all_reviews = []
    
    for page in range(1, num_pages + 1):
        url = f"{base_url}&page={page}"
        print(f"Scraping page {page}: {url}")
        
        response = requests.get(url, headers=headers)
        print(f"Response status code: {response.status_code}")
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract professor information and reviews
        professors = soup.find_all('div', class_='TeacherCard__StyledTeacherCard-syjs0d-0')
        print(f"Number of professors found on page {page}: {len(professors)}")
        
        for professor in professors:
            # ... (rest of the function remains the same)
        
            print(f"Total reviews after page {page}: {len(all_reviews)}")
        
        # Be respectful with request frequency
        time.sleep(random.uniform(1, 3))
    
    return all_reviews


In [66]:
import os
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\hicha\Downloads\professor


In [69]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def search_professor_selenium(name):
    print(f"Searching for professor: {name}")
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode for efficiency
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    search_url = f'https://www.ratemyprofessors.com/search/professors/12013?q={name.replace(" ", "%20")}'
    print(f"Navigating to search URL: {search_url}")
    driver.get(search_url)
    
    try:
        # Wait for search results to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.TeacherCard__StyledTeacherCard-syjs0d-0.dLJIlx'))
        )
        
        # Find the first professor profile link
        profile_elements = driver.find_elements(By.CSS_SELECTOR, 'a.TeacherCard__StyledTeacherCard-syjs0d-0.dLJIlx')
        if not profile_elements:
            print("No profile elements found. Check if the selector is correct.")
            return None
        
        # Extract the href attribute and construct the full URL
        relative_url = profile_elements[0].get_attribute('href')
        profile_url = relative_url
        print(f"Found profile URL: {profile_url}")
        return profile_url
    except Exception as e:
        print(f"Error finding profile link: {e}")
        return None
    finally:
        driver.quit()

def scrape_professor_selenium(url):
    print(f"Scraping professor profile from URL: {url}")
    chrome_options = Options()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(url)
    
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.NameTitle__Name-dowf0z-0'))
        )
        
        # Save page source for debugging
        with open("page_source.html", "w", encoding="utf-8") as file:
            file.write(driver.page_source)
        
        print("Extracting data...")
        
        # Extract professor data
        try:
            first_name = driver.find_element(By.CSS_SELECTOR, 'div.NameTitle__Name-dowf0z-0 > span:first-of-type').text.strip()
        except Exception as e:
            print(f"Error finding first name: {e}")
            first_name = None

        try:
            last_name = driver.find_element(By.CSS_SELECTOR, 'div.NameTitle__Name-dowf0z-0 > span:last-of-type').text.strip()
        except Exception as e:
            print(f"Error finding last name: {e}")
            last_name = None
        
        # Extract similar professors
        try:
            similar_professors_elements = driver.find_elements(By.CSS_SELECTOR, 'span.SimilarProfessorListItem__TeacherNameSpan-x7cr0c-1.gMAFfH')
            similar_professors = [element.text.strip() for element in similar_professors_elements]
        except Exception as e:
            print(f"Error finding similar professors: {e}")
            similar_professors = None
        
        # Print extracted data
        print(f"First Name: {first_name}")
        print(f"Last Name: {last_name}")
        print(f"Similar Professors: {similar_professors}")
        
        return {
            'first_name': first_name,
            'last_name': last_name,
            'similar_professors': similar_professors
        }
    except Exception as e:
        print(f"Error extracting data: {e}")
        return None
    finally:
        driver.quit()

# Example usage
professor_name = 'Michael Beeson'
profile_url = search_professor_selenium(professor_name)
if profile_url:
    data = scrape_professor_selenium(profile_url)
    print("Scraping completed. Result:")
    print(data)



# Example usage
professor_name = 'Michael Beeson'
profile_url = search_professor_selenium(professor_name)
if profile_url:
    data = scrape_professor_selenium(profile_url)
    print("Scraping completed. Result:")
    print(data)


Searching for professor: Michael Beeson
Navigating to search URL: https://www.ratemyprofessors.com/search/professors/12013?q=Michael%20Beeson
Found profile URL: https://www.ratemyprofessors.com/professor/3
Scraping professor profile from URL: https://www.ratemyprofessors.com/professor/3
Extracting data...
First Name: Michael
Last Name: Beeson
Scraping completed. Result:
{'first_name': 'Michael', 'last_name': 'Beeson'}


In [49]:
html = driver.page_source
print(html)  # Check the structure and update selectors as necessary


NameError: name 'driver' is not defined

In [42]:
import requests
from bs4 import BeautifulSoup
import time
import random
import json

def scrape_professor_by_id(professor_id):
    url = f"https://www.ratemyprofessors.com/professor/{professor_id}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return None
    
    soup = BeautifulSoup(response.content, 'html.parser')
    script_tag = soup.find('script', id='__NEXT_DATA__')
    
    if not script_tag:
        return None
    
    try:
        json_data = json.loads(script_tag.string)
        professor_data = json_data['props']['pageProps']['professorData']
        
        name = f"{professor_data['firstName']} {professor_data['lastName']}"
        department = professor_data.get('department', 'N/A')
        school = professor_data.get('school', {}).get('name', 'N/A')
        rating = professor_data.get('avgRating', 'N/A')
        num_ratings = professor_data.get('numRatings', 0)
        
        reviews = []
        for rating in professor_data.get('ratings', [])[:5]:  # Get up to 5 recent reviews
            review_text = rating.get('comment', 'No comment provided.')
            reviews.append(review_text)
        
        return {
            "professor": name,
            "department": department,
            "school": school,
            "rating": rating,
            "num_ratings": num_ratings,
            "reviews": reviews
        }
    except (KeyError, json.JSONDecodeError):
        return None

def scrape_ratemyprofessors(start_id=1, max_professors=3, max_failures=3):
    all_professors = []
    current_id = start_id
    consecutive_failures = 0
    
    while len(all_professors) < max_professors and consecutive_failures < max_failures:
        print(f"Trying professor ID: {current_id}")
        professor_data = scrape_professor_by_id(current_id)
        
        if professor_data:
            all_professors.append(professor_data)
            consecutive_failures = 0
            print(f"Found professor: {professor_data['professor']}")
        else:
            consecutive_failures += 1
        
        current_id += 1
        time.sleep(random.uniform(1, 3))  # Be respectful with request frequency
    
    print(f"Scraped {len(all_professors)} professors")
    return all_professors

# Usage
scraped_data = scrape_ratemyprofessors(start_id=1, max_professors=50, max_failures=100)
print(f"Number of scraped professors: {len(scraped_data)}")
if scraped_data:
    print(f"Sample scraped data: {scraped_data[0]}")
else:
    print("No data scraped")

Trying professor ID: 1
Trying professor ID: 2
Trying professor ID: 3
Trying professor ID: 4
Trying professor ID: 5
Trying professor ID: 6
Trying professor ID: 7
Trying professor ID: 8
Trying professor ID: 9
Trying professor ID: 10
Trying professor ID: 11
Trying professor ID: 12
Trying professor ID: 13
Trying professor ID: 14
Trying professor ID: 15
Trying professor ID: 16
Trying professor ID: 17
Trying professor ID: 18
Trying professor ID: 19
Trying professor ID: 20
Trying professor ID: 21
Trying professor ID: 22
Trying professor ID: 23
Trying professor ID: 24
Trying professor ID: 25
Trying professor ID: 26
Trying professor ID: 27
Trying professor ID: 28
Trying professor ID: 29
Trying professor ID: 30
Trying professor ID: 31
Trying professor ID: 32
Trying professor ID: 33
Trying professor ID: 34
Trying professor ID: 35
Trying professor ID: 36
Trying professor ID: 37
Trying professor ID: 38
Trying professor ID: 39
Trying professor ID: 40
Trying professor ID: 41
Trying professor ID: 42
T

KeyboardInterrupt: 

In [38]:
import json

# Load the JSON data from the file
with open('professors_data.json', 'r') as file:
    scraped_data = json.load(file)

# Process the data
processed_data = [data for data in scraped_data]


In [43]:
print(f"Number of vectors in processed_data: {len(processed_data)}")
print(f"Sample vector: {processed_data[0] if processed_data else 'No data'}")

Number of vectors in processed_data: 60
Sample vector: {'id': 3, 'first_name': 'Michael', 'last_name': 'Beeson', 'similar_professors': ['Wendy Lee', 'Ashish Khanchandani', ''], 'ratings': ['Well... the subject of differentials & linear algebra is hard. That said, he does try to make it as easy as possible by using variation of example problems on the midterms. Sample midterm/ final was just like the real exam. Study, Study, Study. Got by this class by self-learning everything from khan academy and youtube. Textbook is useless', "Math 123 can be one of the hardest courses for engineering students at SJSU. Therefore, I believe that Prof. Beeson has done the best he could. I don't know why people keep complaining about partial credit, but I agree with his point of view - If you want to become an engineer, you are not allowed to make mistakes. That's life, deal with it.", 'Prof. Beeson is extremely apathetic to students, he does not care if you pass or fail. On exams, he does not give any 

In [46]:
from pinecone import Pinecone, ServerlessSpec
from pinecone.exceptions import PineconeApiException
import os

# Load environment variables
load_dotenv()

# Initialize Pinecone
api_key = "b06218ed-b4ea-48db-a318-6e6870d7edd0"
# Ensure correct endpoint and environment
pc = Pinecone(api_key, environment="us-east-1")

try:
    # List existing indexes
    existing_indexes = pc.list_indexes()
    print("Existing indexes:", existing_indexes)

    # Check if the index already exists
    index_names = [idx['name'] for idx in existing_indexes.get('indexes', [])]
    if "rag" not in index_names:
        pc.create_index(
            name="rag",
            dimension=768,  # Use the correct dimension here
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        print("Index 'rag' created successfully.")
    else:
        print("Index 'rag' already exists, skipping creation.")
except PineconeApiException as e:
    # Print the exception details for debugging
    print(f"An error occurred: {e}")
    # Check if the status attribute exists and handle it
    if hasattr(e, 'status') and e.status == 409:
        print("Index 'rag' already exists, no action taken.")
    else:
        raise e


Existing indexes: {'indexes': [{'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'rag-n9979me.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'rag',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}
Index 'rag' already exists, skipping creation.


In [50]:
import os
import google.generativeai as genai
from pinecone import Pinecone

# Configure Gemini API key
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Initialize Pinecone
pinecone_api_key = "b06218ed-b4ea-48db-a318-6e6870d7edd0"  # Replace with your actual API key
pinecone_host = "https://rag-n9979me.svc.aped-4627-b74a.pinecone.io"
pinecone_environment = "us-east-1"  # Ensure this matches your Pinecone environment
pc = Pinecone(api_key=pinecone_api_key, environment=pinecone_environment, host=pinecone_host)

# Reference your Pinecone index
index = pc.Index("rag")  # Replace with your actual index name

data = []  # Data to be upserted
model = 'models/embedding-001'

# Assuming processed_data is a list of dictionaries with professor info
for professor in processed_data:
    for review in professor['ratings']:
        # Skip empty reviews
        if not review.strip():
            print(f"Skipping empty review for professor {professor['id']}")
            continue
        
        try:
            # Generate embedding using Gemini
            embedding_response = genai.embed_content(
                model=model,
                content=review,
                task_type="retrieval_document"
            )
            
            # Check for the embedding key and ensure it contains data
            embedding = embedding_response.get('embedding')
            if not embedding:
                print(f"No embedding returned for review: {review}")
                continue
            
            # Append data to the list for upsert
            data.append({
                "id": f"{professor['id']}_{len(data)}",  # Unique ID for each review
                "values": embedding,
                "metadata": {
                    "first_name": professor["first_name"],
                    "last_name": professor["last_name"],
                    "similar_professors": professor["similar_professors"],
                    "review": review
                }
            })
        except Exception as e:
            print(f"Error generating embedding for review: {review}. Error: {e}")

# Upsert data to Pinecone
try:
    upsert_response = index.upsert(
        vectors=data,  # Ensure this matches the expected input format for Pinecone
        namespace="ns1"
    )
    print(f"Upserted count: {upsert_response.get('upserted_count', 0)}")
except Exception as e:
    print(f"Error upserting data to Pinecone: {e}")


Skipping empty review for professor 157
Upserted count: 598
